##T5파인 튜닝 실습: 뉴스 요약
트랜스포머 인코더로 사전 학습 (Pre‐training) 된 모델이 BERT, 트랜스포머 디코더로 사전 학습된 모델이
GPT 라면, 인코더‐디코더 구조를 유지한 채 사전 학습이 된 모델로 BART 와 T5 가 있습니다. 이번 실습에
서는 T5 를 파인 튜닝하여 한국어 뉴스 요약 실습을 진행해보겠습니다.

###1. 데이터 다운로드
실습을 위해 transformers 패키지와 구글 드라이브로부터 파일을 다운로드 하기 위한 패키지인 gdown
을 설치합니다.

In [1]:
!pip install transformers
!pip install gdown

훈련 데이터와 테스트 데이터를 다운로드해보겠습니다. 이 데이터는 AI Hub 에서 제공하는 데이터셋입
니다.

In [2]:
# 학습 데이터 다운로드
!gdown https://drive.google.com/uc?id=13l621lx2nSnXpFpzh78UUEyds_DAzyn6
# 테스트 데이터 다운로드
!gdown https://drive.google.com/uc?id=10LwhiPlgjOZbtF0Bv5395wYIm23y_QfT

Downloading...
From (original): https://drive.google.com/uc?id=13l621lx2nSnXpFpzh78UUEyds_DAzyn6
From (redirected): https://drive.google.com/uc?id=13l621lx2nSnXpFpzh78UUEyds_DAzyn6&confirm=t&uuid=c519a672-6d0e-423c-885e-2f08fcc8ff8a
To: /content/summ_train.json
100% 1.16G/1.16G [00:09<00:00, 129MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=10LwhiPlgjOZbtF0Bv5395wYIm23y_QfT
From (redirected): https://drive.google.com/uc?id=10LwhiPlgjOZbtF0Bv5395wYIm23y_QfT&confirm=t&uuid=9fdc753c-62bf-41c6-9ace-8cce6da842c9
To: /content/summ_test.json
100% 147M/147M [00:01<00:00, 102MB/s]


각각 summ_train.json, summ_test.json 파일이 다운로드 됩니다. 판다스의 데이터프레임을 이용하여
해당 파일들을 로드해보겠습니다. 실습에서의 학습 시간을 절약하기 위해서 전체 데이터셋 중에서 학습
데이터는 4000 개, 테스트 데이터는 200 개만 사용하겠습니다. 물론, 실제로는 더 많은 데이터셋을 학습
할 수록 성능이 더 좋기 때문에 현업의 서비스에서 사용하실 생각이라면 제공하는 모든 데이터를 활용하
시기 바랍니다.

In [3]:
import pandas as pd

DATA_TRAIN_PATH = 'summ_train.json'
train_df = pd.read_json(DATA_TRAIN_PATH)
train_df = train_df.dropna()
train_df = train_df[:4000]
print('학습 데이터의 개수:', len(train_df))

DATA_TEST_PATH = 'summ_test.json'
test_df = pd.read_json(DATA_TEST_PATH)
test_df = test_df.dropna()
test_df = test_df[:200]
print('테스트 데이터의 개수:', len(test_df))

학습 데이터의 개수: 4000
테스트 데이터의 개수: 200


AI Hub 에서 제공하는 이 데이터는 다소 복잡한 구조를 갖고 있습니다. 학습 데이터와 테스트 데이터 각각
상위 5 개만 출력해봅시다.

In [4]:
train_df.head()

,name,delivery_date,documents
0,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741778', 'category': '종합', 'media_t..."
1,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741792', 'category': '종합', 'media_t..."
2,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741793', 'category': '스포츠', 'media_..."
3,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741794', 'category': '정치', 'media_t..."
4,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741797', 'category': '종합', 'media_t..."


In [5]:
test_df.head()

,name,delivery_date,documents
0,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626877', 'category': '정치', 'media_t..."
1,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626896', 'category': '종합', 'media_t..."
2,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626904', 'category': 'IT,과학', 'medi..."
3,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340627450', 'category': '사회', 'media_t..."
4,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340627465', 'category': '경제', 'media_t..."


BART 학습에 사용하기 위해서는 ‘요약하기 전의 원문’ 과’ 요약문’ 이 두 가지만 있으면 됩니다. 참고로 이 두 개의 텍스트는 위의 데이터프레임에서 ‘documents’ 열에 저장되어져 있습니다. 이를 파싱
하여 원문은’article_original’ 열에, 그리고 요약문은 ‘abstractive’ 열에 저장하는 아래의 전처리 함수
preprocess_data() 를 사용하여 새로운 데이터프레임’train_data’ 와’test_data’ 를 얻어보겠습니다.

In [6]:
def preprocess_data(data):
  outs = []
  for doc in data['documents']:
    line = []
    line.append(doc['media_name'])
    line.append(doc['id'])
    para = []
    for sent in doc['text']:
      for s in sent:
        para.append(s['sentence'])
    line.append(para)
    line.append(doc['abstractive'][0])
    line.append(doc['extractive'])
    a = doc['extractive']
    if a[0] == None or a[1] == None or a[2] == None:
      continue
    outs.append(line)
  outs_df = pd.DataFrame(outs)
  outs_df.columns = ['media', 'id', 'article_original', 'abstractive', 'extractive']
  return outs_df

전처리 후의 학습 데이터를 출력해봅시다.

In [7]:
# 원문과 요약문을 각각 'article_original'와 'abstractive'열에 저장
train_data = preprocess_data(train_df)
train_data.head()

,media,id,article_original,abstractive,extractive
0,광양신문,290741778,"[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,"[2, 3, 10]"
1,광양신문,290741792,"[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,"[2, 4, 11]"
2,광양신문,290741793,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...",전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,"[3, 5, 7]"
3,광양신문,290741794,"[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...","[2, 3, 4]"
4,광양신문,290741797,"[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,"[3, 7, 4]"


In [8]:
# 원문과 요약문을 각각 'article_original'와 'abstractive'열에 저장
test_data = preprocess_data(test_df)
test_data.head()

,media,id,article_original,abstractive,extractive
0,한국경제,340626877,"[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"[0, 1, 3]"
1,한국경제,340626896,"[] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다., 바른미래당이 8일 ...",8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,"[2, 1, 6]"
2,한국경제,340626904,"[[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀., 지난 3일 한국이 세계 최초...",지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,"[1, 5, 8]"
3,한국경제,340627450,[] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,"[0, 1, 2]"
4,한국경제,340627465,"[[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,"[1, 3, 4]"


원문과 요약문이 각각’article_original’ 와’abstractive’ 열에 저장된 데이터프레임을 얻었습니다.
’extractive’ 열의 경우 지금 여기서 하고자하는 생성 요약을 위한 데이터셋이 아니라 BERTSum 과 같
은 추출 요약 모델을 위한 레이블이므로 여기서는 사용하지 않습니다. train_data 의 첫번째 샘플의
article_original 열의 값을 출력해보겠습니다.

In [9]:
# train_data의 첫번째 샘플의 article_original 열의 값 출력
train_data['article_original'].loc[0]


['ha당 조사료 400만원…작물별 차등 지원',
 '이성훈 sinawi@hanmail.net',
 '전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.',
 '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.',
 '올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다.',
 '지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다.',
 '지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다.',
 '농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다.',
 'ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다.',
 '벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다.',
 '논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다.',
 '전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다.',
 '최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.']

위의 결과를 보면 현재 원문이 저장된 article_original 의 경우에는 각 문장을 원소로 하는 파이썬의 리스
트 형태로 저장되어져 있어 이를 하나의 본문으로 저장하여’news’ 열에 저장하고, train_data 의 첫번째
샘플의 news 열의 값을 출력해보겠습니다.

In [10]:
# train_data의 첫번째 샘플의 news 열의 값 출력
train_data['news'] = train_data['article_original'].apply(lambda x: ' '.join(x))
test_data['news'] = test_data['article_original'].apply(lambda x: ' '.join(x))
print(train_data['news'].loc[0])

ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.


이제 우리가 실제로 학습에 사용할 열은 train_data 에서’news’ 열과’abstractive’ 열입니다.

In [11]:
train_data[['news', 'abstractive']].head()

,news,abstractive
0,ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


### 2. 정수 인코딩을 위한 Dataset 생성


In [12]:
import tensorflow as tf
from transformers import TFT5ForConditionalGeneration, T5TokenizerFast
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import CosineDecay
import numpy as np
from tqdm import tqdm

T5 모델에 사용될 입력 데이터와 레이블을 준비하는 데이터셋 클래스를 작성합니다.

In [13]:
import numpy as np
import tensorflow as tf

class T5SummaryDataset(tf.keras.utils.Sequence):
  def __init__(self, df, tokenizer, max_len, batch_size, ignore_index=-100):
    # 데이터셋 초기화
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.docs = df
    self.batch_size = batch_size
    self.ignore_index = ignore_index
    self.indices = list(range(len(self.docs)))

  def __len__(self):
    # 배치 수 반환
    return len(self.indices) // self.batch_size

  def __getitem__(self,idx):
    # 특징 인덱스의 배치 데이터 변환
    if idx >= len(self):
      raise IndexError("Index out of range")

    # 현재 배치의 인덱스 추출
    batch_indices = self.indices[idx * self.batch_size :(idx +1) * self.batch_size]
    batch = self.docs.iloc[batch_indices]

    input_ids = []
    labels = []

    for _, instance in batch.iterrows():
      # 입력 텍스트 인코딩: 'summarize: ' 접두사 추가 후 정수  인코딩
      encoded_input = self.tokenizer.encode("summarize: " +instance['news'],
                                            max_length = self.max_len, padding='max_length',
                                            truncation=True)
      input_ids.append(encoded_input)

      # 레이블(요약) 텍스트 인코딩: 요약 텍스트를 정수 인코딩
      encoded_label = self.tokenizer.encode(instance['abstractive'], max_length= self.max_len,
                                            padding='max_length', truncation=True)
      # 레이블 패딩을 ignore_index(-100)으로 변경
      label = [la if la != self.tokenizer.pad_token_id else self.ignore_index for la in encoded_label]
      labels.append(label)

    # 배치 데이터 반환
    return{
        'input_ids': np.array(input_ids),
        'labels': np.array(labels)
    }

  def on_epoch_end(self):
    # 에포크 종료 시 데이터 인덱스 셔플
    np.random.shuffle(self.indices)

In [14]:
import numpy as np
import tensorflow as tf
class T5SummaryDataset(tf.keras.utils.Sequence):
  def __init__(self, df, tokenizer, max_len, batch_size, ignore_index=-100):
    # 데이터셋 초기화
    self.tokenizer = tokenizer # 토크나이저
    self.max_len = max_len # 최대 시퀀스 길이
    self.docs = df # 데이터 프레임
    self.batch_size = batch_size # 배치크기
    self.ignore_index = ignore_index # 패딩 토큰의 인덱스
    self.indices = list(range(len(self.docs))) # 데이터 인덱스 리스트

  def __len__(self):
    # 배치수 반환
    return len(self.indices) // self.batch_size

  def __getitem__(self, idx):
    # 특정 인덱스의 배치 데이터 반환
    if idx >= len(self):
      raise IndexError("Index out of range")

    # 현재 배치의 인덱스 추출
    batch_indices = self.indices[idx * self.batch_size : (idx + 1) * self.batch_size]
    batch = self.docs.iloc[batch_indices]

    input_ids = []
    labels = []

    for _, instance in batch.iterrows():
      # 입력 텍스트 인코딩: 'summarize: ' 접두사 추가 후 정수  인코딩
      encoded_input = self.tokenizer.encode("summarize: " + instance['news'], max_length=self.max_len, padding='max_length', truncation=True)
      input_ids.append(encoded_input)

      # 레이블(요약) 텍스트 인코딩: 요약 텍스트를 정수 인코딩
      encoded_label = self.tokenizer.encode(instance['abstractive'],max_length=self.max_len, padding='max_length', truncation=True)
      # 레이블 패딩을 ignore_index(-100)으로 변경
      label = [l if l != self.tokenizer.pad_token_id else self.ignore_index for l in encoded_label]
      labels.append(label)

    # 배치 데이터 반환
    return {
        'input_ids': np.array(input_ids),
        'labels': np.array(labels)
        }
  def on_epoch_end(self):
    # 에포크 종료 시 데이터 인덱스 셔플
    np.random.shuffle(self.indices)

T5SummaryDataset 클래스는 텍스트 요약 모델 훈련을 위한 배치 데이터셋을 만듭니다.
init 메소드에서는 필요한 모든 매개변수를 초기화합니다. 데이터프레임, 토크나이저, 최대 길이, 배치 크
기 등을 저장합니다.
getitem 메소드가 이 클래스의 핵심입니다. 각 데이터에 대해 입력 텍스트와 레이블 (요약) 을 처리합니
다. 입력 텍스트 처리 시 “summarize:” 를 앞에 붙이는데, 이는 T5 모델에게 요약 작업을 수행하라고 지시
하는 역할을 합니다. 텍스트는 토크나이저로 인코딩되고, 최대 길이에 맞춰 잘리거나 패딩됩니다. 레이
블 처리에서 주목할 점은 패딩 토큰을 ignore_index(‐100) 로 대체하는 것입니다. 이는 손실 계산 시 패딩
을 무시하게 만들어, 실제 의미가 있는 내용에 대해서만 학습이 이루어지게 합니다.
on_epoch_end 메소드는 학습할 때 각 에포크 종료 시 데이터가 섞이도록 만듭니다. 이를 통해 모델이 매
에포크마다 다른 순서로 데이터를 학습하게 되어, 과적합을 줄이고 일반화 성능을 향상시킬 수 있습니
다.

###3. 모델 클래스 선언
이제 모델을 선언해보겠습니다. TFT5ForConditionalGeneration.from_pretrained 는 텍스트 생성을 위
해 T5 모델을 사용하며, 한국어 텍스트에 특화된 paust/pko‐t5‐base 모델을 사용합니다.

In [15]:
model = TFT5ForConditionalGeneration.from_pretrained('paust/pko-t5-base', from_pt=True)
tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
/usr/local/lib/python3.11/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected

tokenizer_config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

###4. 데이터 로더 변환
이제 학습을 위한 하이퍼파라미터를 설정해봅시다.

In [16]:
# 하이퍼파라미터 설정
batch_size = 4
max_len = 512
lr = 3e-5
max_epochs = 10
warmup_ratio = 0.1


배치 크기는 4, 모델의 입력으로 사용할 데이터의 최대 길이는 512, 학습률 (learning rate) 는 3e‐5 입니다.
이는 0.00003 을 의미합니다. 학습 횟수에 해당하는 max_epochs 의 값은 10, warmup_ratio = 0.1 는 학
습 초기에 학습률을 점진적으로 증가시키는 비율을 나타냅니다. 전체 학습 과정의 10% 동안 학습률이 점
진적으로 증가합니다.

In [17]:
train_dataset = T5SummaryDataset(train_data, tokenizer, max_len=max_len, batch_size=batch_size)
test_dataset = T5SummaryDataset(test_data, tokenizer, max_len=max_len, batch_size=batch_size)

total_steps = len(train_dataset) * max_epochs
warmup_steps = int(total_steps * warmup_ratio)
lr_schedule = CosineDecay(initial_learning_rate=lr, decay_steps=total_steps)
optimizer = Adam(learning_rate=lr_schedule)

total_steps = len(train_dataset)* max_epochs는 전체 학습 과정에서 총 몇 번의 업데이트 (스텝) 가 이루어질지를 계산합니다. 이는 데이터셋의 배치 수와 에포크 수를 곱한 값입니다. 예를 들
어, 데이터셋에 100 개의 배치가 있고, 2 번의 에포크가 있다면 총 200 번의 업데이트가 이루어집니다. warmup_steps = int(total_steps * warmup_ratio)는 학습 초기 단계에서 학습률이 점
진적으로 증가하는 스텝 수를 계산합니다. 예를 들어, 총 200 스텝 중에서 10% 인 20 스텝 동안 학습률이 점진적으로 증가하게 됩니다. lr_schedule = CosineDecay(initial_learning_rate=lr, decay_steps=total_steps)는 Cosine Decay 스케줄을 사용하여 학습률이 학습과정 동안 점진적으로 감소하도록 설정합니다. 학습률은 초기에는 lr로 설정되고, 전체 학습 스텝 동안 점진적으로 감소합니다.
마지막으로, optimizer = Adam(learning_rate=lr_schedule)는 Adam 최적화 알고리즘을 설정합니다. 학습률은 앞에서 정의한 lr_schedule에 따라 변합니다. Adam 최적화 알고리즘은 모멘텀을 이용해 학습 속도를 빠르게 하고, 안정적인 수렴을 가능하게 합니다.
이 코드는 학습 초기 단계에서는 학습률을 점진적으로 증가시키고, 전체 학습 과정에서는 Cosine Decay 스케줄에 따라 학습률을 감소시키면서 모델을 최적화하도록 설정된 것입니다.
배치 크기 단위로 묶인 데이터셋의 크기를 확인해봅시다.

In [18]:
len(train_dataset)

1000

train_dataset 의 길이가 1,000 인데 이는 위에서 학습 데이터를 4,000 개로 지정하였고 데이터를 4 개씩
묶었기 때문입니다.

###5. 데이터 확인
정수 인코딩과 패딩이라는 전처리 전, 후 어떤 차이가 있는지 살펴봅시다. 예를 들어서 정수 인코딩 전의학습 데이터의 첫번째 샘플의 원문은 다음과 같았습니다.

In [19]:
print('첫 샘플의 원문 텍스트: ', train_data['news'].loc[0])

첫 샘플의 원문 텍스트:  ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.


이를 tokenizer.encode() 를 통해 정수 인코딩 후에 최대 길이 512 를 맞추기 위해서 패딩한 결과는 다음과
같습니다.

In [20]:
print('첫 샘플의 원문 텍스트의 정수 인코딩 및 패딩 결과: ', train_dataset[0]['input_ids'][0])
print('정수 인코딩 및 패딩 후의 길이: ', len(train_dataset[0]['input_ids'][0]))

첫 샘플의 원문 텍스트의 정수 인코딩 및 패딩 결과:  [ 7675    78 20359    74 26159    27   222  7231   480   222  1526   541
   222  7004    17 23431    15    15    15 13238   681   222 18165   222
   926   222  5009  1303   222    84  2667    66  7066    33    73  3415
 11261 11866    15 39816   222  5809  6171   278   222  2533   222 12050
   871   333   222  6778   403   373   222  1745   701   222   863   222
  1387   667   222  2642   429   222  2533   222  1951  2487   354   333
   222  2430   222  1808  4210   222   500    15   222  2533   222  1951
  2487   354   274   222  1650   333   222   527  1963   222  1283   279
   222  1650   222  2233   222  4758 13238   824   222  1967   222   450
   222   804   222 13238   291   222 49511   222  1650  2911   222  1928
   222  2470   466   333   222   336  3502  1116   222  2452   267    15
   222  1387   222  4147   302   222  1283   222   804   222 13238   222
  5431   222  1247  4107   311   222  1404   222    22   348  7231   302
   222   585   222  

In [21]:
# 해당 데 이터를 tok‐enizer.decode() 를 통해 복원해봅시다.
print('첫 샘플의 정수 인코딩 후 복원 결과: ')
tokenizer.decode(train_dataset[0]['input_ids'][0])

첫 샘플의 정수 인코딩 후 복원 결과: 


'summarize: ha당 조사료 400만원...작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논</s>'

맨 앞에 summarize:가 부착되어져 있고 원문 내용이 중간에 끊긴 채 마지막에 </s</s>>가 부착되어져 있습니다. 정수 1 이 </s</s>>에 해당되는 것입니다. 이렇게 원문이 잘리는 경우는 원칙적으로는 학습에 사용하지 않는 것이 좋습니다. 하지만 여기서는 별도로 빼지않고 학습에 사용하겠습니다. 실제 적용 시에는
입력이 잘린 데이터는 가급적 사용하지 말아주세요. 이번에는 원문이 아니라 요약문을 가지고 확인해봅
시다.

In [22]:
print('첫 샘플의 요약문: ', train_data['abstractive'].loc[0])

첫 샘플의 요약문:  전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .


이제 요약문의 정수 인코딩 결과를 확인해봅시다. 요약문에서는 패딩을 ‐100 으로 진행합니다.

In [23]:
print('첫 샘플의 요약문의 정수 인코딩 및 -100으로 패딩한 결과')
train_dataset[0]['labels'][0]

첫 샘플의 요약문의 정수 인코딩 및 -100으로 패딩한 결과


array([ 5809,  6171,   278,   222,  2533,   222, 12050,   871,   333,
         222,  6778,   403,   373,   222,  1745,   701,   222,   863,
         222,  1387,   667,   222,  1650,   333,   222,   527,  1963,
         222,  1283,   279,   222,  1650,   222,  2233,   222,  4758,
       13238,   824,   222,  1967,   222,   450,   222,   804,   222,
       13238,   291,   222, 49511,   222,  1650,  2911,   222,  1928,
         222,  2470,   466,   333,   222,   336,  3502,  1116,   222,
           8,  2533,   222,  1951,  2487,   354,     8,   333,   222,
        2430,   403,   373,   222,  2642,   701,   293,   222,   443,
         222,  1243,   222,  3858,   349,   667,   222,    19,   515,
         222,  3732,   349,   579,   222, 14205,   222,  2500,   284,
         222, 30492,  1588,   402,   389,   222,  1143,  6604,   222,
          15,     1,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
        -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
        -100,  -100,

정수 인코딩이 되면서 텍스트가 정수 시퀀스로 변경되었을 뿐만 아
니라 뒤에 숫자 ‐100 이 연속해서 등장합니다. 최대 길이를 512 로 설정하였으므로 길이를 512 로 맞추기
위해서 패딩된 것입니다. 이번에는 레이블을 정수 인코딩 후 다시 텍스트로 복원한 결과를 확인해봅시
다.

In [24]:
# -100의 값은 tokenizer_decode하면 에러나므로 임시로 0으로 변경 후 출력
test_array = train_dataset[0]['labels'][0]
test_array[test_array == -100] = 0
print('첫 샘플의 요약문 레이블 :', tokenizer.decode(test_array))

첫 샘플의 요약문 레이블 : 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

이제 학습시 어텐션 마스크를 적용하기 위해서 어텐션 마스크를 구현해봅시다.


In [25]:
# 어텐션 마스크 생성 함수
def create_attention_mask(input_ids):
  # input_ids와 패딩 토큰 ID를 비교하여 불리언 마스크 생성
  # 패딩이 아닌 토큰은 True, 패딩 토큰은 False가 됩니다.
  boolean_mask = tf.not_equal(input_ids, tokenizer.pad_token_id)
  return tf.cast(boolean_mask, tf.float32)

먼저 create_attention_mask 함수는 입력 시퀀스에 대한 어텐션 마스크를 생성합니다. 이 함수는 입력
ID 와 토크나이저의 패딩 토큰 ID 를 비교하여 패딩이 아닌 토큰 위치에는 1 을, 패딩 토큰 위치에는 0 을 할
당합니다. 이렇게 생성된 마스크는 모델이 패딩 토큰을 무시하고 실제 입력 토큰에만 주의를 기울이도록
합니다.

### 7. 학습
이제 T5 모델을 학습시켜봅시다. 아래 코드는 각 에폭마다 훈련 손실과 검증 손실을 계산합니다. 검증 손
실이 개선되면 모델을 저장하고, 에포크 종료 시 데이터셋을 셔플합니다. 이 과정을 최대 8 번 반복하여 최
적의 모델을 찾습니다.

In [ ]:
 # 훈 련 루 프
best_loss = float('inf')

# 총 훈 련 할 에 폭 수 설 정. 이 는 예 시 값 으 로 , 실 제 사 용 시 조 정 이 필 요 할 수 있 습 니 다.
max_epochs = 8

for epoch in range(max_epochs):
  print(f'에 포 크 {epoch+1}/{max_epochs}')
  # 훈 련
  total_train_loss = 0.0
  for batch in tqdm(train_dataset, total=len(train_dataset), desc="훈 련 중"):
    attention_mask = create_attention_mask(batch['input_ids'])
    with tf.GradientTape() as tape:
      outputs = model(input_ids=batch['input_ids'], # 정 수 인 코 딩
                      attention_mask=attention_mask, # 어 텐 션 마 스 크
                      labels=batch['labels'], # 레 이 블 의 정 수 인 코 딩
                      training=True) # 훈 련 모 드 활 성 화
      loss = outputs.loss # 모 델 이 반 환 한 손 실 값
    total_train_loss += tf.reduce_mean(loss).numpy() # 배 치 의 평 균 손 실 을 누 적
    # 그 래 디 언 트 계 산
    gradients = tape.gradient(loss, model.trainable_variables)
    # 계 산 된 그 래 디 언 트 를 사 용 하 여 모 델 파 라 미 터 업 데 이 트
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  avg_train_loss = total_train_loss / len(train_dataset) # 에 폭 의 평 균 훈 련 손 실 계 산
  print(f'훈 련 손 실: {avg_train_loss:.4f}')

  # 평 가
  total_val_loss = 0.0

  for batch in tqdm(test_dataset, total=len(test_dataset), desc="검 증 중"):
    attention_mask = create_attention_mask(batch['input_ids'])
    outputs = model(input_ids=batch['input_ids'],
                    attention_mask=attention_mask,
                    labels=batch['labels'],
                    training=False)

    total_val_loss += tf.reduce_mean(outputs.loss).numpy() # 검 증 배 치 의 평 균 손 실 누 적

  avg_val_loss = total_val_loss / len(test_dataset) # 에 폭 의 평 균 검 증 손 실 계 산
  print(f'검 증 손 실: {avg_val_loss:.4f}')

  # 최 고 성 능 모 델 저 장
  # 현 재 에 폭 의 검 증 손 실 이 이 전 최 저 손 실 보 다 낮 은 경 우
  if avg_val_loss < best_loss: # 최 저 손 실 업 데 이 트
    best_loss = avg_val_loss
    model.save_pretrained('model') # 현 재 모 델 상 태 를 파 일 로 저 장
    print(f'검증 손실이 {best_loss:.4f}로 개선 되었습니다. 체크포인트를 저장 했습니다.')
  # 에 포 크 종 료 시 데 이 터 셔 플
  train_dataset.on_epoch_end()
print("훈 련 이 완 료 되 었 습 니 다.")

에 포 크 1/8


훈 련 중:  13%|█▎        | 128/1000 [05:28<37:01,  2.55s/it]

In [ ]:
# 모 데 로 드
loaded_model = TFT5ForConditionalGeneration.from_pretrained('model')

In [ ]:
def evaluate_model(model, test_dataset):
  # 총 검 증 손 실 을 저 장 할 변 수 초 기 화
  total_val_loss = 0.0

  # test_dataset의 각 배 치 에 대 해 반 복
  for batch in tqdm(test_dataset, total=len(test_dataset), desc="평 가 중"):
    # 어 텐 션 마 스 크 생 성
    attention_mask = create_attention_mask(batch['input_ids'])

    # 모 델 에 입 력 을 전 달 하 고 출 력 얻 기
    outputs = model(input_ids=batch['input_ids'],
                    attention_mask=attention_mask,
                    labels=batch['labels'],
                    training=False) # 평 가 모 드 로 설 정

    # 배 치 의 평 균 손 실 을 계 산 하 고 총 손 실 에 더 함
    total_val_loss += tf.reduce_mean(outputs.loss).numpy()

  # 전 체 데 이 터 셋 에 대 한 평 균 검 증 손 실 계 산
  avg_val_loss = total_val_loss / len(test_dataset)

  # 평 균 검 증 손 실 반 환
  return avg_val_loss